In [1]:
from ProcessData import *
from keras.models import load_model
model = load_model('model.h5', compile=True)

In [2]:
y_preds = (model.predict(preprocess_data("Earn money, just click here!")) > 0.5).astype("int")[0][0]

1/1 [==============================] - 0s 334ms/step


In [3]:
y_preds

1